In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score as acc
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
import math
from sklearn import model_selection

In [2]:
# Defining modle metrics
from sklearn.metrics import r2_score
def R2(y_true, y_pred):
    return r2_score(y_true, y_pred)
from sklearn.metrics import mean_squared_log_error
def MSLE(y_true, y_pred):
    return mean_squared_log_error(y_true, y_pred)
from sklearn.metrics import explained_variance_score
def EX_VAR(y_true, y_pred):
    return explained_variance_score(y_true, y_pred)
from sklearn.metrics import mean_absolute_error
def MAE(y_true, y_pred):
    return mean_absolute_error(y_true, y_pred)
from sklearn.metrics import mean_squared_error
def RMSE(y_true, y_pred):
    return mean_squared_error(y_true, y_pred,squared=True)
def MAP(y_true, y_pred):
    error=0
    for yt, yp in zip(y_true, y_pred):
        error += (yt - yp) / (yt)
    return (error / len(y_true))*100


In [3]:
#Reading the data and droping a useless column and droping all rows with Nan values in any column
#Scenario V-Modified.xlsx
DataSet = pd.read_excel( 'Dataset.xlsx').drop(columns=['name','DOI','content (%)', 'concrete elastic modulus(GPa)',
                                                          'Ductility index(deltap/deltay)','steel fiber type' ]).dropna()
# If necesasry yse the followings:

# Making one hot encoding of a column
#data = pd.get_dummies(data=DataSet, columns=['steel fiber type'])
#data = data.drop(columns = ['Soil Type_Fissured'])
#data = data[data['Soil Type_Intact']>0]


# adding a new column with name Kfold, later we will use it 
data = DataSet.copy()

# scaling
scaler = MinMaxScaler()
scaler.fit(data)
data = pd.DataFrame(scaler.transform(data.values), index=data.index, columns=data.columns)

data['kfold'] = -1
print(data.columns)
print(len(data))
data.head(5)

Index(['fiber aspect ratio(mm/mm)', 'fiber density(g/cm3)',
       'volume fraction(%)', 'reinforcement ratio(%)', 'beam width(mm)',
       'beam height(mm)', 'beam length(mm)', 'shear span-depth ratio',
       'fiber tensile strength(MPa)', 'Ductility index(deltau/deltay)',
       'Max Load', 'Deltap', 'kfold'],
      dtype='object')
184


C:\Users\Fabagh001\Anaconda3\envs\FARAMARZ\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


,fiber aspect ratio(mm/mm),fiber density(g/cm3),volume fraction(%),reinforcement ratio(%),beam width(mm),beam height(mm),beam length(mm),shear span-depth ratio,fiber tensile strength(MPa),Ductility index(deltau/deltay),Max Load,Deltap,kfold
0,0.000,0.0,0.000000,0.132394,0.25,0.171429,0.571429,0.683333,0.000000,0.390661,0.058660,0.522806,-1
1,0.000,0.0,0.000000,0.211268,0.25,0.171429,0.571429,0.683333,0.000000,0.224125,0.098821,0.402627,-1
2,0.650,1.0,0.666667,0.132394,0.25,0.171429,0.571429,0.683333,0.927787,0.159533,0.086762,0.153214,-1
3,0.650,1.0,0.666667,0.211268,0.25,0.171429,0.571429,0.683333,0.927787,0.145525,0.128629,0.107881,-1
4,0.975,1.0,0.666667,0.132394,0.25,0.171429,0.571429,0.683333,0.831947,0.159144,0.093588,0.164952,-1


In [4]:
#Random shuffling the data 
df = data.sample(frac= 1, random_state=10).reset_index(drop=True)

#Adding numbers for each fold of data to the Kfold column 
kf = model_selection.KFold(n_splits=5)
for fold, (trn_, val_) in enumerate(kf.split(X=df)):
    df.loc[val_, 'kfold'] = fold

# check the Kfold column, now we can select a scpecific chunk of data with using Kfold column
df.head(3)

,fiber aspect ratio(mm/mm),fiber density(g/cm3),volume fraction(%),reinforcement ratio(%),beam width(mm),beam height(mm),beam length(mm),shear span-depth ratio,fiber tensile strength(MPa),Ductility index(deltau/deltay),Max Load,Deltap,kfold
0,0.00,0.000000,0.000000,0.126761,0.125,0.214286,0.714286,0.516667,0.407654,0.052529,0.086989,0.090833,0
1,0.45,0.987342,0.666667,0.098592,0.250,0.071429,0.200000,0.258333,0.366057,0.112840,0.076181,0.036333,0
2,0.64,0.987342,0.253333,0.021127,0.700,0.285714,0.771429,0.321667,0.447587,0.140078,0.032948,0.127446,0


In [5]:
list_R2_train, list_R2_test = [] ,[]
list_RMSE_train, list_RMSE_test = [],[]
list_MAE_train, list_MAE_test = [] ,[]
list_MPE_train, list_MPE_test = [] ,[]
list_MAP_train, list_MAP_test =[] ,[]

for i in range(5):
    # calling a fold and getting the X and Y for that fold
    test_fold = i
    x_train = df[df['kfold']!= test_fold].drop(columns=['Deltap','kfold'])
    y_train = df[df['kfold']!= test_fold]['Deltap']
    x_test = df[df['kfold']== test_fold].drop(columns=['Deltap','kfold'])
    y_test = df[df['kfold']== test_fold]['Deltap']

    # calling the model and hyper parameters 
    GB=GradientBoostingRegressor(min_samples_split=4, min_samples_leaf=6,n_estimators=3000,learning_rate=0.001,max_depth=6)
    #GB=GradientBoostingRegressor()
    GB.fit(x_train,y_train)
    
    #predicting via model
    y_pred_train = GB.predict(x_train)
    y_pred_test = GB.predict(x_test)
    
    
    # getting model metrics for each fold and storing them in lists
    list_R2_train.append(R2(y_train,y_pred_train.flatten()))
    list_R2_test.append(R2(y_test,y_pred_test.flatten()))
    
    list_RMSE_train.append(RMSE(y_train,y_pred_train.flatten()))
    list_RMSE_test.append(RMSE(y_test,y_pred_test.flatten()))
    
    list_MAE_train.append(MAE(y_train,y_pred_train.flatten()))
    list_MAE_test.append(MAE(y_test,y_pred_test.flatten()))



In [6]:
print('GBM')
print('  ')
print('%%% TEST %%%')
print('R2 = ',round(np.mean(list_R2_test),2))
print('RMSE = ',round(np.mean(list_RMSE_test),2))
print('MAE = ',round(np.mean(list_MAE_test),2))

#print(np.mean(list_R2_test))
print('  ')
print('*** TRAIN ***')
print('R2 = ',round(np.mean(list_R2_train),2))
print('RMSE = ',round(np.mean(list_RMSE_train),2))
print('MAE = ',round(np.mean(list_MAE_train),3))

#print(np.mean(list_R2_test))

GBM
  
%%% TEST %%%
R2 =  0.76
RMSE =  0.01
MAE =  0.05
  
*** TRAIN ***
R2 =  0.95
RMSE =  0.0
MAE =  0.023


In [7]:
list_R2_train, list_R2_test = [] ,[]
list_RMSE_train, list_RMSE_test = [],[]
list_MAE_train, list_MAE_test = [] ,[]
list_MPE_train, list_MPE_test = [] ,[]
list_MAP_train, list_MAP_test =[] ,[]

for i in range(5):
    # calling a fold and getting the X and Y for that fold
    test_fold = i
    x_train = df[df['kfold']!= test_fold].drop(columns=['Max Load','kfold'])
    y_train = df[df['kfold']!= test_fold]['Max Load']
    x_test = df[df['kfold']== test_fold].drop(columns=['Max Load','kfold'])
    y_test = df[df['kfold']== test_fold]['Max Load']

    # calling the model and hyper parameters 
    #GB=GradientBoostingRegressor(min_samples_split=4, min_samples_leaf=6,n_estimators=3000,learning_rate=0.001,max_depth=6)
    GB=GradientBoostingRegressor()
    GB.fit(x_train,y_train)
    
    #predicting via model
    y_pred_train = GB.predict(x_train)
    y_pred_test = GB.predict(x_test)
    
    
    # getting model metrics for each fold and storing them in lists
    list_R2_train.append(R2(y_train,y_pred_train.flatten()))
    list_R2_test.append(R2(y_test,y_pred_test.flatten()))
    
    list_RMSE_train.append(RMSE(y_train,y_pred_train.flatten()))
    list_RMSE_test.append(RMSE(y_test,y_pred_test.flatten()))
    
    list_MAE_train.append(MAE(y_train,y_pred_train.flatten()))
    list_MAE_test.append(MAE(y_test,y_pred_test.flatten()))



In [8]:
print('GBM')
print('  ')
print('%%% TEST %%%')
print('R2 = ',round(np.mean(list_R2_test),2))
print('RMSE = ',round(np.mean(list_RMSE_test),2))
print('MAE = ',round(np.mean(list_MAE_test),2))

#print(np.mean(list_R2_test))
print('  ')
print('*** TRAIN ***')
print('R2 = ',round(np.mean(list_R2_train),2))
print('RMSE = ',round(np.mean(list_RMSE_train),2))
print('MAE = ',round(np.mean(list_MAE_train),3))

#print(np.mean(list_R2_test))

GBM
  
%%% TEST %%%
R2 =  0.96
RMSE =  0.0
MAE =  0.02
  
*** TRAIN ***
R2 =  1.0
RMSE =  0.0
MAE =  0.007


In [9]:
list_R2_train, list_R2_test = [] ,[]
list_RMSE_train, list_RMSE_test = [],[]
list_MAE_train, list_MAE_test = [] ,[]
list_MPE_train, list_MPE_test = [] ,[]
list_MAP_train, list_MAP_test =[] ,[]

for i in range(5):
    # calling a fold and getting the X and Y for that fold
    test_fold = i
    x_train = df[df['kfold']!= test_fold].drop(columns=['Ductility index(deltau/deltay)','kfold'])
    y_train = df[df['kfold']!= test_fold]['Ductility index(deltau/deltay)']
    x_test = df[df['kfold']== test_fold].drop(columns=['Ductility index(deltau/deltay)','kfold'])
    y_test = df[df['kfold']== test_fold]['Ductility index(deltau/deltay)']

    # calling the model and hyper parameters 
    GB=GradientBoostingRegressor(min_samples_split=4, min_samples_leaf=6,n_estimators=3000,learning_rate=0.001,max_depth=6)
    #GB=GradientBoostingRegressor()
    GB.fit(x_train,y_train)
    
    #predicting via model
    y_pred_train = GB.predict(x_train)
    y_pred_test = GB.predict(x_test)

    
    # getting model metrics for each fold and storing them in lists
    list_R2_train.append(R2(y_train,y_pred_train.flatten()))
    list_R2_test.append(R2(y_test,y_pred_test.flatten()))
    
    list_RMSE_train.append(RMSE(y_train,y_pred_train.flatten()))
    list_RMSE_test.append(RMSE(y_test,y_pred_test.flatten()))
    
    list_MAE_train.append(MAE(y_train,y_pred_train.flatten()))
    list_MAE_test.append(MAE(y_test,y_pred_test.flatten()))
    


In [10]:
print('GBM')
print('  ')
print('%%% TEST %%%')
print('R2 = ',round(np.mean(list_R2_test),2))
print('RMSE = ',round(np.mean(list_RMSE_test),2))
print('MAE = ',round(np.mean(list_MAE_test),2))

#print(np.mean(list_R2_test))
print('  ')
print('*** TRAIN ***')
print('R2 = ',round(np.mean(list_R2_train),2))
print('RMSE = ',round(np.mean(list_RMSE_train),2))
print('MAE = ',round(np.mean(list_MAE_train),3))

#print(np.mean(list_R2_test))

GBM
  
%%% TEST %%%
R2 =  0.43
RMSE =  0.01
MAE =  0.07
  
*** TRAIN ***
R2 =  0.86
RMSE =  0.0
MAE =  0.032
